# Oppgaver med datasettet Games
Datasettet games inneholder data om 80000 brettspill. Dette er data som navn på spillet, hvilken rating det har fått av brukerene, spilletid, etc. Disse oppgavene bygger på en bloggpost, og du kan lese denne posten og mer om datasettet [her](https://www.dataquest.io/blog/machine-learning-python/). Merk at fasit for oppgaven også finnes på denne lenken (i tillegg til i git).

## Forberedelser

### Last ned data
Vi starter med å laste ned data. Datasettet finnes på stien https://github.com/ThaWeatherman/scrapers/raw/master/boardgamegeek/games.csv og skal lagres i filen games.csv.

Filen kan finnes under files i menyen til venstre.

In [0]:
!wget -q https://github.com/ThaWeatherman/scrapers/raw/master/boardgamegeek/games.csv -O games.csv

### Importer biblioteker
Vi må nå legge til Pyton-bibliotekene som vi trenger.

In [0]:
# For prosessering av data
from sklearn.decomposition import PCA
# Maskinlæringsalgoritmen som vi bruker
from sklearn.cluster import KMeans
# For å lage diagrammer bruker vi pyplot.
import matplotlib.pyplot as plt
# Pandas har bibliotek for å lese csv-filer
from pandas import read_csv
# For å kunne dele datasettet i to ulike sett, ett for trening og ett for test
from sklearn.model_selection import train_test_split
# For å kunne bruke lineær regresjon
from sklearn.linear_model import LinearRegression
# For å kunne finne ut hvor bra modellen vår gjør det
from sklearn.metrics import mean_squared_error

## Datasettet
Videre skal vi ta en kikk på datasettet.

### Innlesing
Vi starter med å lese inn datasettet.

In [0]:
games = read_csv("games.csv")

### Undersøkelse og preprosessering
Videre kan vi se hvilke data vi har. Hvilke kolonner har vi?

In [0]:
print(games.columns)

Deretter må vi sjekke hvor mye data vi har. Det kan vi gjøre ved se på størrelsen til matrisen.

In [0]:
print(games.shape)

Dette betyr at det finnes totalt 81312 spill i dagagrunnlaget, der det finnes 20 datapunkter om hvert spill.

Vi kan også undersøke hva som ligger i matrisen, ved å se på et lite utvalg av dataene.

In [0]:
print(games.head())

Nå kan det være interessant å se litt statistikk. Hvordan er de ulike ratingene. Vi kan lage et histogram over 'average_rating' og vise dette.

In [0]:
plt.hist(games['average_rating'])
plt.show

Det viser seg at det er en del spill som har rating 0. Disse spillene har ikke blitt bedømt av noen, og kan derfor fjernes. Vi vil derfor sette matrisen games til å være lik seg selv, men der antall brukere som har ratet spillet er større enn 0. Fyll inn kode for dette under.

In [0]:
games = games[games['users_rated'] > 0]

Hvor mange rader står vi igjen med da?

In [0]:
print(games.shape)

(56932, 20)


Videre kan vi fjerne de radene der det mangler verdier.

In [0]:
games = games.dropna(axis = 0)

Hvor mange rader gjenstår nå?

In [0]:
print(games.shape)

(56894, 20)


## Læring
Vi skal nå se om vi kan lære noe av datasettet. Her skal vi prøve oss på klustering. Det går ut på å plassere data som ligner på hverandre i klynger, slik at det er liten forskjell på spill innenfor samme klynge.

###KMeans modell
Vi lager en modell med funksjonene KMeans. Her kan vi bruke 5 klynger og setter random_state til 1. Grunne til dette er at det skal være mulig å reprodusere resultatet senere.

In [0]:
kmeans_model = KMeans(n_clusters=5, random_state=1)

Klustering fungerer best på kolonner med tall. Derfor henter vi ut alle kolonnene som er numeriske. Kanskje metoden \_get_numeric_data() kan brukes?

In [0]:
good_columns = games._get_numeric_data()

Deretter trener vi opp algoritmen. Det kan gjøres med metoden fit på modellen. Husk at du kun skal bruke kolonnene med tall fra over.

In [0]:
kmeans_model.fit(good_columns)

Resultatet av klustering blir plassert i etikketer. Disse kan vi hente ut fra modellen, og representere de ulike guppene algoritmen har funnet.

In [0]:
labels = kmeans_model.labels_

Siden vi har så mange kolonner, er det vanskelig å visualisere dataene. For å kunne gjøre dette om til noe vi klarer å visulaisere, må vi redusere dataene til færre dimmensjoner. PCA står for prinicipal componenet analysis, og blir brukt for å redusere dimensjoner uten at man mister for mye av sammenhengene i gruppene. Vi reduserer til 2 dimmensjoner, slik at det er mulig å lage et plot av gruppene.

In [0]:
pca_2 = PCA(2)
plot_columns = pca_2.fit_transform(good_columns)

Til slutt lager vi et diagram med de ulike klyngene.

In [0]:
plt.scatter(x=plot_columns[:, 0], y = plot_columns[:, 1], c=labels)
plt.show()

## Forutsi rating
Vi vil gjerne kunne forutsi hva som blir 'average_rating' for et spill

### Finne korrelasjoner
Korrelasjoner sier noe om hvilken sammenheng data har. Dersom to ulike datatyper korrelerer godt sammen, kan man bruke den ene datatypen til å forutsi den neste.

Siden vi skal foruts 'average_rating', kan vi finne ut hvilke typer data som korrelerer med dette.

In [0]:
games.corr()['average_rating']

Her kan vi se at 'average_rating' korrelerer helt med seg selv. Dette er slik det skal være. I tillegg har også id en høy korrelasjon. Grunnen til dette kan være at eldre spill får dårligere rating, siden nye spill kanskje er mer komplekse og dermed får en bedre score.

Nå skal vi velge hvilke kolonner som skal være med i prediksjonen. Som nevnt tidligere er det vanskelig å bruke kolonner som ikke er numeriske til prediksjone. I tillegg må vi fjerne alle kolonner som er et resultat av rating. Siden vi skal predikere rating, har vi altså ikke tilgang til en del av kolonnene som er laget med utgangspunkt i rating.

Først kan vi hente ut alle kolonnene og liste opp disse.

In [0]:
columns = games.columns.tolist()
print(columns)

Videre fjerne vi kolonner som vi ikke skal bruke, altså kolonnene vi skal predikere, de som inneholder tekst, og de som blir laget som et resultat av rating.

In [0]:
columns = [c for c in columns if c not in ["average_rating", "bayes_average_rating", "type", "name"]]

Vi lagrer også det vi vil predikere i en variabel.

In [0]:
target = "average_rating"

### Trening og test
For å kunne finne ut hvor bra algoritmen vår gjør det, må vi dele datasettet i to, ett for trening og ett for test. Grunnen til dette er at vi ønsker å evaluere algoritmen på data den ikke har sett tidliger. Vi deler derfor datasettet vårt inn i et treningssett som består av 80% av dataen, og et testsett som består av 20%. For å kunne gjenskape resultatet setter vi random_state til 1.

In [0]:
train = games.sample(frac=0.8, random_state=1)
test = games.loc[~games.index.isin(train.index)]

Vi sjekker størrelsen på de ulike settene.

In [0]:
print(train.shape)
print(test.shape)

Lineær regresjon passer fint dersom variablene man bruker for prediksjon og målet (target) har en korrelasjon. Dette så vi i stad, og vi prøver derfor å bruke en lineær regresjonsmodell.

In [0]:
model = LinearRegression()

Videre bruker vi modellen på treningsdataene og sier hvilken kolonne vi skal forutse.

In [0]:
model.fit(train[columns], train[target])

Vi skal nå lage prediksjoner for testsettet.

In [0]:
predictions = model.predict(test[columns])

For å finne ut hvor bra algoritmen gjør det, kan sjekke mean squared error (MSE). Dette vil vise hvor langt fra den riktig prediksjonen algoritmen er, i gjennomsnitt.

In [0]:
mean_squared_error(predictions, test[target])

## Random Forest
Vi har nå prøvd oss på lineær regresjon. Videre kan vi teste det samme ved å bruke en annen model, nemlig random forest. Denne klarer også å finne sammenhenger i datasettet som ikke er lineære.

In [0]:
# Importer random forest
from sklearn.ensemble import RandomForestRegressor

# Initialiser modellen
model = RandomForestRegressor(n_estimators=100, min_samples_leaf=10, random_state=1)

# Bruk modellen på treningsdata
model.fit(train[columns], train[target])

# Gjør prediksjoner
predictions = model.predict(test[columns])

# Sjekk feilen
mean_squared_error(predictions, test[target])

## Forslag til videre arbeid

- Prøv support vector machine (SVM)
- Prøv å slå sammen flere algoritmer for å få bedre resultat
- Prøv å forutsi en annen kolonne, slik som 'average_weight'
- Lag features basert på de ulike tekskolonnen, slik som lengden på navnet til spillet, antall ord, etc.